In [ ]:
import numpy as np
import pandas as pd
import os

df = pd.read_csv('archive/historical-tornado-tracks.csv', sep=';')
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

Poniższa mapa przedstawia pewien trend przesuwania się tornad na wschód. W zależności od doboru parametrów mag ten trend jest inny. Dla słabszych tornad ten trend jest bardziej wyraźny.

In [ ]:
import folium

# PRZYGOTWANIE DANYCH
df1 = df[['yr', 'slat', 'slon', 'mag']]

# Wybranie stosunkowo silnych tornad - zgodnie ze skalą Enhanced Fujity EF1 - oznacza umiarkowane zniszczenia
# (w data secie nie ma informacji czy korzystamy z F czy EF) - założyłem że podali EF bo tego się używa teraz
data_groups = {
    "Wszystkie znaczące (F1-F5)": df[df['mag'] >= 1].copy(),
    "Umiarkowane (F1-F2)": df[(df['mag'] >= 1) & (df['mag'] <= 2)].copy(),
    "Bardzo mocne (F3-F5)": df[df['mag'] >= 3].copy()
}

# Centroidy dla każdej grupy
centroids_dict = {}
for name, data in data_groups.items():
    data['decade'] = (data['yr'] // 10) * 10
    grp_centroid = data.groupby('decade')[['slat', 'slon']].mean().reset_index()
    centroids_dict[name] = grp_centroid

# RYSOWANIE MAPY
base_centroids = centroids_dict["Wszystkie znaczące (F1-F5)"]
avg_lat = base_centroids['slat'].mean()
avg_lon = base_centroids['slon'].mean()

m1 = folium.Map(location=[avg_lat, avg_lon], zoom_start=6, tiles='cartodbpositron')

for name, centroids in centroids_dict.items():
    fg = folium.FeatureGroup(name=name, show=(name == "Wszystkie znaczące (F1-F5)"))

    points = []
    for idx, row in centroids.iterrows():
        decade = int(row['decade'])
        color = 'blue' if decade < 1980 else 'red'
        folium.CircleMarker(
            location=[row['slat'], row['slon']],
            radius=8,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            popup=f"<b>Warstwa: {name}</b><br>Dekada: {decade}s",
            tooltip=f"{name}: {decade}s"
        ).add_to(fg)

        if decade == 1950:
            folium.Marker(
                [row['slat'], row['slon']],
                icon=folium.Icon(color='blue', icon='play', prefix='fa'),
                tooltip=f"POCZĄTEK ANALIZY ({decade})"
            ).add_to(fg)

        if decade == 2010:
            folium.Marker(
                [row['slat'], row['slon']],
                icon=folium.Icon(color='red', icon='stop', prefix='fa'),
                tooltip=f"KONIEC ANALIZY ({decade})"
            ).add_to(fg)

        points.append([row['slat'], row['slon']])

    weight = 3
    folium.PolyLine(points, color="#333333", weight=weight, opacity=0.6, dash_array='5, 5').add_to(fg)
    fg.add_to(m1)


folium.LayerControl(position='topright', collapsed=False).add_to(m1)

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 230px; height: 160px;
    border:2px solid grey; z-index:9999; font-size:14px;
    background-color:white; opacity: 0.9;
    padding: 10px; border-radius: 10px;
    box-shadow: 3px 3px 5px rgba(0,0,0,0.3);
    ">
    <b>Legenda: Oś Czasu</b><br>
    <hr style="margin: 5px 0;">
    <i class="fa fa-circle" style="color:blue"></i> Lata 1950 - 1979<br>
    <small>(Starsze dane)</small><br>

    <i class="fa fa-circle" style="color:red"></i> Lata 1980 - 2013<br>
    <small>(Nowsze dane)</small><br>

    <i class="fa fa-minus" style="color:black; font-weight:bold"></i> Ścieżka przesunięcia<br>
    <br>
</div>
'''

title_html = '''
     <h3 align="center" style="font-size:20px; font-family: Arial, sans-serif;">
     <b>Analiza wielowarstwowa: Przesunięcie centroidów tornad</b>
     </h3>
     '''

m1.get_root().html.add_child(folium.Element(legend_html))
m1.get_root().html.add_child(folium.Element(title_html))

# Dopasowanie widoku do bazy
m1.fit_bounds(base_centroids[['slat', 'slon']].values.tolist())

m1.save('mapa_1_layers_final.html')
m1

Poniżej mapka z clustrami gdzie pokazujemy najsilniejsze tornada, czyli skala EF4 i EF5 - możemy zamiast tego pokazać np tylko tornada z ofiarami i rannymi (chodzi mi troche o ograniczenie danych)

In [ ]:
from folium.plugins import MarkerCluster

df_extreme = df[(df['mag'] == 4) | (df['mag'] == 5)]

m2 = folium.Map(location=[37, -95], zoom_start=4, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(m2)

for idx, row in df_extreme.iterrows():
    color = 'red' if row['mag'] == 5 else 'orange'

    # Popup z informacjami - zastanawiałem się, bo albo możemy zrobić po kliknięciu, albo po najechaniu i nie wiem jak lepiej
    popup_text = f"""
    <b>Data:</b> {row['yr']}-{row['mo']}-{row['dy']}<br>
    <b>Siła:</b> EF{row['mag']}<br>
    <b>Stan:</b> {row['state']}<br>
    <b>Ofiary śmiertelne:</b> {row['fat']}<br>
    <b>Ranni:</b> {row['inj']}
    """

    folium.CircleMarker(
        location=[row['slat'], row['slon']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        popup=folium.Popup(popup_text, max_width=200),
        tooltip=f"EF{row['mag']} ({row['yr']})"
    ).add_to(marker_cluster)


title_html = '''
     <h3 align="center" style="color: black; font-size:20px"><b>Lokalizacja tornad katastroficznych (EF4-EF5)</b></h3>
     '''
m2.get_root().html.add_child(folium.Element(title_html))

m2.save('mapa_3_extreme_tornadoes.html')
m2

Kolejna mapka to slider gdzie pokazujemy np tornada w kolejnych latach

In [ ]:
from folium.plugins import TimestampedGeoJson

# Filtrujemy tylko silne tornada (EF3, EF4, EF5), żeby mapa była czytelna i lekka
df_slider = df[df['mag'] >= 3].copy()

features = []
for idx, row in df_slider.iterrows():
    if row['mag'] == 5:
        color = 'red'
    elif row['mag'] == 4:
        color = 'orange'
    elif row['mag'] == 3:
        color = 'green'
    else:
        color = 'blue'

    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['slon'], row['slat']],
        },
        'properties': {
            'time': row['date'],
            'popup': f"<b>Data:</b> {row['date']}<br><b>Siła:</b> EF{row['mag']}<br><b>Stan:</b> {row['state']}",
            'icon': 'circle',
            'iconstyle': {
                'fillColor': color,
                'fillOpacity': 0.8,
                'stroke': 'false',
                'radius': 6
            },
            'style': {'color': color}
        }
    }
    features.append(feature)

timestamped_geojson = TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='P1Y',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True,
    duration='P1Y'
)


m_slider = folium.Map(location=[38, -95], zoom_start=5, tiles='cartodbpositron')
timestamped_geojson.add_to(m_slider)

title_html = '''
     <h3 align="center" style="font-size:18px"><b>Historia silnych tornad (EF3+) - Oś Czasu</b></h3>
     '''
m_slider.get_root().html.add_child(folium.Element(title_html))

legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index:9999; font-size:14px;
     background-color:white; padding: 10px; border: 1px solid grey; border-radius: 5px;">
     <b>Legenda</b><br>
     <i class="fa fa-circle" style="color:red"></i> EF5 (Katastrofalne)<br>
     <i class="fa fa-circle" style="color:orange"></i> EF4<br>
     <i class="fa fa-circle" style="color:blue"></i> EF3
</div>
'''
m_slider.get_root().html.add_child(folium.Element(legend_html))

# Zapis
m_slider.save('mapa_4_time_slider.html')
m_slider

Kolejna mapka to wizualizacja tornad o różnych porach dnia (w dzień albo w nocy) - dzięki temu będziemy wiedzieli o jakie środki ostrzegawcze najlepiej się wzmocnić